<a href="https://colab.research.google.com/github/gpdigitalworld/intrinsic-value-dashboard/blob/main/DCF_Intrinsic_NIFTYmidcap_150.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Install required packages
!pip install yfinance pandas openpyxl

import yfinance as yf
import pandas as pd
import time
from IPython.display import FileLink

# ----------------------------
# STEP 1: Load Nifty Midcap 150 List
# ----------------------------
nifty_midcap_url = "https://archives.nseindia.com/content/indices/ind_niftymidcap150list.csv"
midcap_df = pd.read_csv(nifty_midcap_url)
tickers = midcap_df['Symbol'].tolist()

# ----------------------------
# STEP 2: DCF Valuation Function
# ----------------------------
def calculate_intrinsic_value(fcf, growth_rate, discount_rate=0.10, terminal_growth_rate=0.04, years=5):
    total_value = 0
    for year in range(1, years + 1):
        fcf *= (1 + growth_rate)
        discounted_fcf = fcf / ((1 + discount_rate) ** year)
        total_value += discounted_fcf
    terminal_value = fcf * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    discounted_terminal_value = terminal_value / ((1 + discount_rate) ** years)
    total_value += discounted_terminal_value
    return total_value

# ----------------------------
# STEP 3: Run DCF Scan with Filters
# ----------------------------
undervalued = []

print("🔍 Scanning Nifty Midcap 150 stocks...\n")
for symbol in tickers:
    try:
        stock = yf.Ticker(f"{symbol}.NS")
        info = stock.info

        # Collect key data
        price = info.get("currentPrice")
        fcf = info.get("freeCashflow")
        growth = info.get("earningsGrowth")  # as decimal
        roe = info.get("returnOnEquity")     # as decimal
        debt_to_equity = info.get("debtToEquity")

        # Apply filters
        if not all([price, fcf, growth, roe, debt_to_equity]):
            continue
        if roe < 0.15 or debt_to_equity > 0.5 or fcf <= 0 or growth <= 0:
            continue

        # Run DCF
        intrinsic = calculate_intrinsic_value(fcf, growth)

        if intrinsic > price:
            undervalued.append({
                "Symbol": symbol,
                "Company": info.get("shortName"),
                "Price": round(price, 2),
                "Intrinsic Value": round(intrinsic, 2),
                "Upside (%)": round((intrinsic - price) / price * 100, 2),
                "ROE": round(roe * 100, 2),
                "Debt to Equity": round(debt_to_equity, 2)
            })

    except Exception as e:
        print(f"⚠️ Error fetching {symbol}: {e}")

    time.sleep(1)  # Avoid rate limiting

# ----------------------------
# STEP 4: Save and Download
# ----------------------------
if undervalued:
    result_df = pd.DataFrame(undervalued)
    file_path = "nifty_midcap_undervalued_filtered.xlsx"
    result_df.to_excel(file_path, index=False)
    print("\n✅ Scan complete. Download your results below:")
    display(FileLink(file_path))
else:
    print("❌ No undervalued midcap stocks found that meet your criteria.")


🔍 Scanning Nifty Midcap 150 stocks...

❌ No undervalued midcap stocks found that meet your criteria.
